# Example full chip design

In [1]:
%load_ext autoreload
%autoreload 2

# Import Modules

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

In [3]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors

from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.couplers.line_tee import LineTee

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

from collections import OrderedDict

from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

## Define "TransmonPocket_sqnl"

In [4]:
from qiskit_metal.qlibrary.core import BaseQubit
import numpy as np
class TransmonPocket_sqnl(BaseQubit):
    default_options = Dict(
        pad_gap='30um',
        inductor_width='20um',
        pad_width='455um',
        pad_height='90um',
        pocket_width='650um',
        pocket_height='650um',
        # 90 has dipole aligned along the +X axis,
        # while 0 has dipole aligned along the +Y axis
        _default_connection_pads=Dict(
            connector_type='0',  # 0 = Claw type, 1 = T-shape
            pad_width='125um',
            pad_height='30um',
            pad_cpw_shift='0um',
            pad_cpw_extent='25um',
            claw_length='30um',
            claw_width='10um',
            claw_gap='6um',
            claw_cpw_length='40um',
            claw_cpw_width='10um',
            ground_spacing='5um',
            connector_location= '0'  # 0~5 means six locations with rotated CCW starting from right-top site
        ))
    """Default drawing options"""

    component_metadata = Dict(short_name='Pocket_claw',
                              _qgeometry_table_path='True',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True')
    """Component metadata"""

    TOOLTIP = """The base `TransmonPocket_sqnl` class."""

    def make(self):
        """Define the way the options are turned into QGeometry.

        The make function implements the logic that creates the geoemtry
        (poly, path, etc.) from the qcomponent.options dictionary of
        parameters, and the adds them to the design, using
        qcomponent.add_qgeometry(...), adding in extra needed
        information, such as layer, subtract, etc.
        """
        self.make_pocket()
        self.make_connection_pads()

    def make_pocket(self):
        """Makes standard transmon in a pocket."""

        # self.p allows us to directly access parsed values (string -> numbers) form the user option
        p = self.p

        # extract chip name
        chip = p.chip

        # since we will reuse these options, parse them once and define them as varaibles
        pad_width = p.pad_width
        pad_height = p.pad_height
        pad_gap = p.pad_gap

        # make the pads as rectangles (shapely polygons)
        pad = draw.rectangle(pad_width, pad_height)
        pad_top = draw.translate(pad, 0, +(pad_height + pad_gap) / 2.)
        pad_bot = draw.translate(pad, 0, -(pad_height + pad_gap) / 2.)

        rect_jj = draw.LineString([(0, -pad_gap / 2), (0, +pad_gap / 2)])
        # the draw.rectangle representing the josephson junction
        # rect_jj = draw.rectangle(p.inductor_width, pad_gap)

        rect_pk = draw.rectangle(p.pocket_width, p.pocket_height)

        # Rotate and translate all qgeometry as needed.
        # Done with utility functions in Metal 'draw_utility' for easy rotation/translation
        # NOTE: Should modify so rotate/translate accepts qgeometry, would allow for
        # smoother implementation.
        polys = [rect_jj, pad_top, pad_bot, rect_pk]
        polys = draw.rotate(polys, p.orientation, origin=(0, 0))
        polys = draw.translate(polys, p.pos_x, p.pos_y)
        [rect_jj, pad_top, pad_bot, rect_pk] = polys

        # Use the geometry to create Metal qgeometry
        self.add_qgeometry('poly',
                           dict(pad_top=pad_top, pad_bot=pad_bot),
                           chip=chip)
        self.add_qgeometry('poly',
                           dict(rect_pk=rect_pk),
                           subtract=True,
                           chip=chip)
        # self.add_qgeometry('poly', dict(
        #     rect_jj=rect_jj), helper=True)
        self.add_qgeometry('junction',
                           dict(rect_jj=rect_jj),
                           width=p.inductor_width,
                           chip=chip)

    def make_connection_pads(self):
        """Makes standard transmon in a pocket."""
        for name in self.options.connection_pads:
            self.make_connection_pad(name)

    def make_connection_pad(self, name: str):
        """Makes n individual connector.

        Args:
            name (str) : Name of the connector
        """
        # self.p allows us to directly access parsed values (string -> numbers) form the user option
        p = self.p
        pc = self.p.connection_pads[name]  # parser on connector options
        # extract chip name
        chip = p.chip
    
        if pc.connector_type == 0:  # Claw connector
            
            # define commonly used variables once
            pad_width = p.pad_width
            pad_height = p.pad_height
            pad_gap = p.pad_gap
            pocket_width = p.pocket_width
            pocket_height = p.pocket_height
    
            c_g = pc.claw_gap
            c_l = pc.claw_length
            c_w = pc.claw_width
            c_c_w = pc.claw_cpw_width
            c_c_l = pc.claw_cpw_length
            g_s = pc.ground_spacing
            con_loc = pc.connector_location
    
            claw_cpw = draw.box(-c_w, -c_c_w / 2, -c_c_l - c_w, c_c_w / 2)
            if con_loc == 0 or  con_loc == 2 or  con_loc == 3 or  con_loc == 5:
                t_claw_height = 2*c_g + 2 * c_w + pad_height  # temp value
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, c_g)
            elif con_loc == 1 or  con_loc == 4:
                t_claw_height = 2*c_g + 2 * c_w + pad_width  # temp value
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, g_s)
            else:
                print('Not appropriate position of connector')
            
            port_line = draw.LineString([(- c_w, 0),
                                     (-c_c_l - c_w, 0)])

            claw_rotate = 0
            if con_loc == 0:
                claw_rotate = -180
            elif con_loc == 1:
                claw_rotate = -90
            elif con_loc == 2:
                claw_rotate = 0
            elif con_loc == 3:
                claw_rotate = 0
            elif con_loc == 4:
                claw_rotate = 90
            elif con_loc == 5:
                claw_rotate = 180
    
            claw_translate_x = (con_loc==0 or con_loc==5)*(pad_width/2 + c_g) + (con_loc==2 or con_loc==3)*(-pad_width/2 - c_g)
            claw_translate_y = (con_loc==0 or con_loc==2)*(pad_height + pad_gap)/2 + (con_loc==5 or con_loc==3)*(-pad_height - pad_gap)/2 + (con_loc==1)*(pad_height + pad_gap/2 + c_g) - (con_loc==4)*(pad_height + pad_gap/2 + c_g)
            
            # Rotates and translates the connector polygons (and temporary port_line)
            polys = [connector_arm, connector_etcher, port_line]
            polys = draw.rotate(polys, claw_rotate, origin=(0, 0))
            polys = draw.translate(polys, claw_translate_x, claw_translate_y)
            polys = draw.rotate(polys, p.orientation, origin=(0, 0))
            polys = draw.translate(polys, p.pos_x, p.pos_y)
            [connector_arm, connector_etcher, port_line] = polys
    
            # Generates qgeometry for the connector pads
            self.add_qgeometry('poly', {f'{name}_connector_arm': connector_arm},
                               chip=chip)
            self.add_qgeometry('poly',
                               {f'{name}_connector_etcher': connector_etcher},
                               subtract=True,
                               chip=chip)
    
            self.add_pin(name, points=port_line.coords, width=c_c_w,input_as_norm=True)
        elif pc.connector_type == 1:  # T-shape connector
            
            # define commonly used variables once
            pad_width = p.pad_width
            pad_height = p.pad_height
            pad_gap = p.pad_gap
            pocket_width = p.pocket_width
            pocket_height = p.pocket_height
    
            c_g = pc.claw_gap
            c_l = 0
            c_w = pc.claw_width
            c_c_w = pc.claw_cpw_width
            c_c_l = pc.claw_cpw_length
            g_s = pc.ground_spacing
            con_loc = pc.connector_location
    
            claw_cpw = draw.box(-c_w, -c_c_w / 2, -c_c_l - c_w, c_c_w / 2)
            if con_loc == 0 or  con_loc == 2 or  con_loc == 3 or  con_loc == 5:
                t_claw_height = pc.t_claw_height
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, c_g)
            elif con_loc == 1 or  con_loc == 4:
                t_claw_height = pc.t_claw_height
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, c_g)
            else:
                print('Not appropriate position of connector')
            
            port_line = draw.LineString([(- c_w, 0),
                                     (-c_c_l - c_w, 0)])

            claw_rotate = 0
            if con_loc == 0:
                claw_rotate = -180
            elif con_loc == 1:
                claw_rotate = -90
            elif con_loc == 2:
                claw_rotate = 0
            elif con_loc == 3:
                claw_rotate = 0
            elif con_loc == 4:
                claw_rotate = 90
            elif con_loc == 5:
                claw_rotate = 180
    
            claw_translate_x = (con_loc==0 or con_loc==5)*(pad_width/2 + g_s) + (con_loc==2 or con_loc==3)*(-pad_width/2 - g_s)
            claw_translate_y = (con_loc==0 or con_loc==2)*(pad_height + pad_gap)/2 + (con_loc==5 or con_loc==3)*(-pad_height - pad_gap)/2 + (con_loc==1)*(pad_height + pad_gap/2 + g_s) - (con_loc==4)*(pad_height + pad_gap/2 + g_s)
            
            # Rotates and translates the connector polygons (and temporary port_line)
            polys = [connector_arm, connector_etcher, port_line]
            polys = draw.rotate(polys, claw_rotate, origin=(0, 0))
            polys = draw.translate(polys, claw_translate_x, claw_translate_y)
            polys = draw.rotate(polys, p.orientation, origin=(0, 0))
            polys = draw.translate(polys, p.pos_x, p.pos_y)
            [connector_arm, connector_etcher, port_line] = polys
    
            # Generates qgeometry for the connector pads
            self.add_qgeometry('poly', {f'{name}_connector_arm': connector_arm},
                               chip=chip)
            self.add_qgeometry('poly',
                               {f'{name}_connector_etcher': connector_etcher},
                               subtract=True,
                               chip=chip)
    
            self.add_pin(name, points=port_line.coords, width=c_c_w,input_as_norm=True)

        else:
            print('Incorrect connector type')
            
        

## Define "LineTee_sqnl"

In [5]:
from qiskit_metal.qlibrary.core import QComponent
class LineTee_sqnl(QComponent):
    """Generates a three pin (+) structure comprised of a primary two pin CPW
    transmission line, and a secondary one pin neighboring CPW transmission
    line that is capacitively coupled to the primary. Such a structure can be
    used, as an example, for generating CPW resonator hangars off of a
    transmission line. (0,0) represents the center position of the component.

    Inherits QComponent class.

    ::

                  (0,0)
        +--------------------------+
                    |
                    |
                    |
                    |
                    |
                    +

    .. image::
        LineTee.png

    .. meta::
        Line Tee

    Options:
        * prime_width: '10um' -- The width of the trace of the two pin CPW transmission line
        * prime_gap: '6um' -- The dielectric gap of the two pin CPW transmission line
        * second_width: '10um' -- The width of the trace of the one pin CPW transmission line
        * second_gap: '6um' -- The dielectric gap of the one pin CPW transmission line (also for the capacitor)
        * t_length: '50um' -- The length for the t branches
        * C_box_width: '50um'
        * C_box_height: '20um'
        * C_box_gap: '20um'
        * C_gap: '5um'
    """
    component_metadata = Dict(short_name='cpw', _qgeometry_table_path='True')
    """Component metadata"""

    #Currently setting the primary CPW length based on the coupling_length
    #May want it to be it's own value that the user can control?
    default_options = Dict(prime_width='10um',
                           prime_gap='6um',
                           second_width='10um',
                           second_gap='6um',
                           t_length='50um')
    """Default connector options"""

    TOOLTIP = """Generates a three pin (+) structure comprised of a primary two pin CPW
    transmission line, and a secondary one pin neighboring CPW transmission
    line that is capacitively coupled to the primary."""

    def make(self):
        """Build the component."""
        p = self.p
        prime_cpw_length = p.t_length * 2
        C_box_width = p.C_box_width
        C_box_height = p.C_box_height
        C_box_gap = p.C_box_gap
        C_gap = p.C_gap
        
        #Primary CPW
        prime_cpw = draw.LineString([[-prime_cpw_length / 2, 0],
                                     [prime_cpw_length / 2, 0]])

        #Secondary CPW
        second_cpw = draw.LineString([[0, -p.prime_width / 2], [0,
                                                                -p.t_length]])
        
        #Capacitor
        C_box = draw.box(-C_box_width/2,-p.t_length-C_box_height,C_box_width/2,-p.t_length)
        C_gap = draw.box(-C_box_width/2-C_box_gap,-p.t_length-C_box_height-C_gap,C_box_width/2+C_box_gap,-p.t_length+C_box_gap)
        
        #Rotate and Translate
        c_items = [prime_cpw, second_cpw]
        c_items = draw.rotate(c_items, p.orientation, origin=(0, 0))
        c_items = draw.translate(c_items, p.pos_x, p.pos_y)
        [prime_cpw, second_cpw] = c_items
        
        C_boxes = [C_box,C_gap]
        C_boxes = draw.rotate(C_boxes, p.orientation, origin=(0, 0))
        C_boxes = draw.translate(C_boxes, p.pos_x, p.pos_y)
        [C_box,C_gap] = C_boxes
        
        #Add to qgeometry tables
        self.add_qgeometry('path', {'prime_cpw': prime_cpw},
                           width=p.prime_width,
                           layer=p.layer)
        self.add_qgeometry('path', {'prime_cpw_sub': prime_cpw},
                           width=p.prime_width + 2 * p.prime_gap,
                           subtract=True,
                           layer=p.layer)
        self.add_qgeometry('path', {'second_cpw': second_cpw},
                           width=p.second_width,
                           layer=p.layer)
        self.add_qgeometry('path', {'second_cpw_sub': second_cpw},
                           width=p.second_width + 2 * p.second_gap,
                           subtract=True,
                           layer=p.layer)
        
        self.add_qgeometry('poly', {'C_box': C_box},
                           layer=p.layer)
        self.add_qgeometry('poly', {'C_gap': C_gap},
                           subtract=True,
                           layer=p.layer)

        #Add pins
        prime_pin_list = prime_cpw.coords
        second_pin_list = second_cpw.coords

        self.add_pin('prime_start',
                     points=np.array(prime_pin_list[::-1]),
                     width=p.prime_width,
                     input_as_norm=True)
        self.add_pin('prime_end',
                     points=np.array(prime_pin_list),
                     width=p.prime_width,
                     input_as_norm=True)
        self.add_pin('second_end',
                     points=np.array(second_pin_list),
                     width=p.second_width,
                     input_as_norm=True)


## Chip dimensions

In [6]:
design = metal.designs.DesignPlanar()

gui = metal.MetalGUI(design)

In [7]:
design.overwrite_enabled = True
design.chips.main

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '9mm',
  'size_y': '6mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

In [8]:
design.chips

{'main': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '0.0mm',
   'size_x': '9mm',
   'size_y': '6mm',
   'size_z': '-750um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}}}

In [9]:
design.chips.main.size.size_x = '10mm'
design.chips.main.size.size_y = '10mm'

# Qubits

In [10]:
TransmonPocket_sqnl.get_template_options(design)

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'connection_pads': {},
 '_default_connection_pads': {'connector_type': '0',
  'pad_width': '125um',
  'pad_height': '30um',
  'pad_cpw_shift': '0um',
  'pad_cpw_extent': '25um',
  'claw_length': '30um',
  'claw_width': '10um',
  'claw_gap': '6um',
  'claw_cpw_length': '40um',
  'claw_cpw_width': '10um',
  'ground_spacing': '5um',
  'connector_location': '0'},
 'pad_gap': '30um',
 'inductor_width': '20um',
 'pad_width': '455um',
 'pad_height': '90um',
 'pocket_width': '650um',
 'pocket_height': '650um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'aedt_q3d_wire_bonds': False,
 'aedt_hfss_wire_bonds': False,
 'hfss_inductance': '10nH',
 'hfss_capacitance': 0,
 'hfss_resistance': 0,
 'hfss_mesh_kw_jj': 7e-06,
 'q3d_inductance': '10nH',
 'q3d_capacitance': 0,
 'q3d_resistance': 0,
 'q3d_mesh_kw_jj': 7e-06,
 'gds_cell_name': 'my_other_junction',
 'aedt_q3d_inductance': 1e-08,
 'aedt_q3d_c

In [11]:
options_1 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    orientation = 0,
    connection_pads=dict(
        readout1 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='25um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '0'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        readout2 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='10um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '2'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        control = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '30um',
                       ground_spacing='100um',
                       claw_width='30um',
                       claw_gap='18um',
                       claw_cpw_length='100um',
                       claw_cpw_width='30um',
                       connector_location=
                       '1'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) 
    ))

options_2 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    orientation = 90,
    connection_pads=dict(
        readout1 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='25um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '0'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        readout2 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='10um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '2'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        control = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '30um',
                       ground_spacing='100um',
                       claw_width='30um',
                       claw_gap='18um',
                       claw_cpw_length='100um',
                       claw_cpw_width='30um',
                       connector_location=
                       '1'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) 
    ))

options_3 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    orientation = 180,
    connection_pads=dict(
        readout1 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='25um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '0'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        readout2 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='10um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '2'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        control = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '30um',
                       ground_spacing='100um',
                       claw_width='30um',
                       claw_gap='18um',
                       claw_cpw_length='100um',
                       claw_cpw_width='30um',
                       connector_location=
                       '1'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) 
    ))

options_4 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    orientation = 270,
    connection_pads=dict(
        readout1 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='25um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '0'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        readout2 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '400um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='10um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '2'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        control = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '30um',
                       ground_spacing='100um',
                       claw_width='30um',
                       claw_gap='18um',
                       claw_cpw_length='100um',
                       claw_cpw_width='30um',
                       connector_location=
                       '1'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) 
    ))

In [12]:
q_1 = TransmonPocket_sqnl(design,'Q_1', options = dict(
        pos_x='0mm', 
        pos_y='2.5mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        **options_1))

q_2 = TransmonPocket_sqnl(design,'Q_2', options = dict(
        pos_x='-2.5mm', 
        pos_y='0mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        **options_2))

q_3 = TransmonPocket_sqnl(design,'Q_3', options = dict(
        pos_x='0mm', 
        pos_y='-2.5mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        **options_3))

q_4 = TransmonPocket_sqnl(design,'Q_4', options = dict(
        pos_x='2.5mm', 
        pos_y='0mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        **options_4))

# gui.rebuild()
# gui.autoscale()

# Resoantors

## Launch Pads

In [13]:
launch_TL_read_11 = LaunchpadWirebond(design, 
                                   'launch_TL_read_11', 
                                   options = dict(
                                       pos_x = '2.5mm', 
                                       pos_y ='4.3mm', 
                                       orientation = '270',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_12 = LaunchpadWirebond(design, 
                                   'launch_TL_read_12', 
                                   options = dict(
                                       pos_x = '0mm', 
                                       pos_y ='4.3mm', 
                                       orientation = '270',
                                       trace_gap = '18um',
                                       trace_width = '30um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_13 = LaunchpadWirebond(design, 
                                   'launch_TL_read_13', 
                                   options = dict(
                                       pos_x = '-2.5mm', 
                                       pos_y ='4.3mm', 
                                       orientation = '270',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))

launch_TL_read_21 = LaunchpadWirebond(design, 
                                   'launch_TL_read_21', 
                                   options = dict(
                                       pos_x = '-4.3mm', 
                                       pos_y ='2.5mm', 
                                       orientation = '0',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_22 = LaunchpadWirebond(design, 
                                   'launch_TL_read_22', 
                                   options = dict(
                                       pos_x = '-4.3mm', 
                                       pos_y ='0mm', 
                                       orientation = '0',
                                       trace_gap = '18um',
                                       trace_width = '30um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_23 = LaunchpadWirebond(design, 
                                   'launch_TL_read_23', 
                                   options = dict(
                                       pos_x = '-4.3mm', 
                                       pos_y ='-2.5mm', 
                                       orientation = '0',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))

launch_TL_read_31 = LaunchpadWirebond(design, 
                                   'launch_TL_read_31', 
                                   options = dict(
                                       pos_x = '-2.5mm', 
                                       pos_y ='-4.3mm', 
                                       orientation = '90',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_32 = LaunchpadWirebond(design, 
                                   'launch_TL_read_32', 
                                   options = dict(
                                       pos_x = '0mm', 
                                       pos_y ='-4.3mm', 
                                       orientation = '90',
                                       trace_gap = '18um',
                                       trace_width = '30um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_33 = LaunchpadWirebond(design, 
                                   'launch_TL_read_33', 
                                   options = dict(
                                       pos_x = '2.5mm', 
                                       pos_y ='-4.3mm', 
                                       orientation = '90',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))

launch_TL_read_41 = LaunchpadWirebond(design, 
                                   'launch_TL_read_41', 
                                   options = dict(
                                       pos_x = '4.3mm', 
                                       pos_y ='-2.5mm', 
                                       orientation = '180',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_42 = LaunchpadWirebond(design, 
                                   'launch_TL_read_42', 
                                   options = dict(
                                       pos_x = '4.3mm', 
                                       pos_y ='0mm', 
                                       orientation = '180',
                                       trace_gap = '18um',
                                       trace_width = '30um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))
launch_TL_read_43 = LaunchpadWirebond(design, 
                                   'launch_TL_read_43', 
                                   options = dict(
                                       pos_x = '4.3mm', 
                                       pos_y ='2.5mm', 
                                       orientation = '180',
                                       trace_gap = '10um',
                                       trace_width = '20um',
                                       taper_height = '375um',
                                       pad_width='250um',
                                       pad_height='300um',
                                       pad_gap = '100um'
                                   ))

# gui.rebuild()
# gui.autoscale()

In [14]:
CapNInterdigital.get_template_options(design)

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'north_width': '10um',
 'north_gap': '6um',
 'south_width': '10um',
 'south_gap': '6um',
 'cap_width': '10um',
 'cap_gap': '6um',
 'cap_gap_ground': '6um',
 'finger_length': '20um',
 'finger_count': '5',
 'cap_distance': '50um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'aedt_q3d_wire_bonds': False,
 'aedt_hfss_wire_bonds': False}

## IDCs

### Definition

In [15]:
from qiskit_metal.qlibrary.core import QComponent
class CapNInterdigital_sqnl(QComponent):
    
    component_metadata = Dict(short_name='cpw',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_path='True')
    """Component metadata"""

    #Currently setting the primary CPW length based on the coupling_length
    #May want it to be it's own value that the user can control?
    default_options = Dict(north_width='10um',
                           north_gap='6um',
                           south_width='10um',
                           south_gap='6um',
                           cap_width='10um',
                           cap_gap='6um',
                           cap_gap_ground='6um',
                           finger_length='20um',
                           finger_count='5',
                           cap_distance='50um',
                           taper_length='200um'
                          )
    """Default connector options"""

    TOOLTIP = """Generates a two pin (+) structure
     comprised of a north CPW transmission line, 
     and a south transmission line, coupled 
     together via a finger capacitor."""

    def make(self):
        """Build the component."""
        p = self.p
        N = int(p.finger_count)

        #Finger Capacitor
        cap_box = draw.Polygon([[(N * p.cap_width + (N - 1) * p.cap_gap)/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [p.north_width/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-p.north_width/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-p.south_width/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [p.south_width/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [(N * p.cap_width + (N - 1) * p.cap_gap)/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2]
                               ])
        make_cut_list = []
        make_cut_list.append([0, (p.finger_length) / 2])
        make_cut_list.append([(p.cap_width) + (p.cap_gap / 2),
                              (p.finger_length) / 2])
        flip = -1

        for i in range(1, N):
            make_cut_list.append([
                i * (p.cap_width) + (2 * i - 1) * (p.cap_gap / 2),
                flip * (p.finger_length) / 2
            ])
            make_cut_list.append([
                (i + 1) * (p.cap_width) + (2 * i + 1) * (p.cap_gap / 2),
                flip * (p.finger_length) / 2
            ])
            flip = flip * -1

        cap_cut = draw.LineString(make_cut_list).buffer(p.cap_gap / 2,
                                                        cap_style=2,
                                                        join_style=2)
        cap_cut = draw.translate(cap_cut,
                                 -(N * p.cap_width + (N - 1) * p.cap_gap) / 2,
                                 0)

        cap_body = draw.subtract(cap_box, cap_cut)
        cap_body = draw.translate(
            cap_body, 0, -p.cap_distance -
            (p.cap_gap + 2 * p.cap_width + p.finger_length) / 2)

        cap_etch = draw.Polygon([[(N * p.cap_width + (N - 1) * p.cap_gap)/2+p.cap_gap_ground, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [p.north_width/2+p.north_gap, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-p.north_width/2-p.north_gap, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2-p.cap_gap_ground, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2-p.cap_gap_ground, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-p.south_width/2-p.south_gap, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [p.south_width/2+p.south_gap, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [(N * p.cap_width + (N - 1) * p.cap_gap)/2+p.cap_gap_ground, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2]
                               ])

        cap_etch = draw.translate(
            cap_etch, 0, -p.cap_distance -
            (p.cap_gap + 2 * p.cap_width + p.finger_length) / 2)

        #CPW
        north_cpw = draw.LineString([[0, p.taper_length], [0, p.taper_length-p.cap_distance]])

        south_cpw = draw.LineString(
            [[
                0, -p.taper_length-p.cap_distance -
                (p.cap_gap + 2 * p.cap_width + p.finger_length)
            ],
             [
                 0, -p.taper_length-2 * p.cap_distance -
                 (p.cap_gap + 2 * p.cap_width + p.finger_length)
             ]])

        #Rotate and Translate
        c_items = [north_cpw, south_cpw, cap_body, cap_etch]
        c_items = draw.rotate(c_items, p.orientation, origin=(0, 0))
        c_items = draw.translate(c_items, p.pos_x, p.pos_y)
        [north_cpw, south_cpw, cap_body, cap_etch] = c_items

        #Add to qgeometry tables
        self.add_qgeometry('path', {'north_cpw': north_cpw},
                           width=p.north_width,
                           layer=p.layer)
        self.add_qgeometry('path', {'north_cpw_sub': north_cpw},
                           width=p.north_width + 2 * p.north_gap,
                           layer=p.layer,
                           subtract=True)

        self.add_qgeometry('path', {'south_cpw': south_cpw},
                           width=p.south_width,
                           layer=p.layer)
        self.add_qgeometry('path', {'south_cpw_sub': south_cpw},
                           width=p.south_width + 2 * p.south_gap,
                           layer=p.layer,
                           subtract=True)

        self.add_qgeometry('poly', {'cap_body': cap_body}, layer=p.layer)
        self.add_qgeometry('poly', {'cap_etch': cap_etch},
                           layer=p.layer,
                           subtract=True)

        #Add pins
        north_pin_list = north_cpw.coords
        south_pin_list = south_cpw.coords

        self.add_pin('north_end',
                     points=np.array(north_pin_list[::-1]),
                     width=p.north_width,
                     input_as_norm=True)
        self.add_pin('south_end',
                     points=np.array(south_pin_list),
                     width=p.south_width,
                     input_as_norm=True)

### Draw

In [16]:
highC_PF_TL_11 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_11', 
                               options = dict(
                                   pos_x = '2.5mm',
                                   pos_y = '3.9mm',
                                   orientation = '0',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '44um', 
                                   finger_length   = '200um', 
                                   finger_count   = '8', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))

highC_PF_TL_13 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_13', 
                               options = dict(
                                   pos_x = '-2.5mm',
                                   pos_y = '3.9mm',
                                   orientation = '0',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '44um', 
                                   finger_length   = '200um', 
                                   finger_count   = '8', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))

highC_PF_TL_21 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_21', 
                               options = dict(
                                   pos_x = '-3.9mm',
                                   pos_y = '2.5mm',
                                   orientation = '90',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '44um', 
                                   finger_length   = '200um', 
                                   finger_count   = '8', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))

highC_PF_TL_23 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_23', 
                               options = dict(
                                   pos_x = '-3.9mm',
                                   pos_y = '-2.5mm',
                                   orientation = '90',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '44um', 
                                   finger_length   = '200um', 
                                   finger_count   = '8', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))

highC_PF_TL_31 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_31', 
                               options = dict(
                                   pos_x = '-2.5mm',
                                   pos_y = '-3.9mm',
                                   orientation = '180',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '44um', 
                                   finger_length   = '200um', 
                                   finger_count   = '8', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))

highC_PF_TL_33 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_33', 
                               options = dict(
                                   pos_x = '2.5mm',
                                   pos_y = '-3.9mm',
                                   orientation = '180',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '44um', 
                                   finger_length   = '200um', 
                                   finger_count   = '8', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))

highC_PF_TL_41 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_41', 
                               options = dict(
                                   pos_x = '3.9mm',
                                   pos_y = '-2.5mm',
                                   orientation = '270',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '44um', 
                                   finger_length   = '200um', 
                                   finger_count   = '8', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))

highC_PF_TL_43 = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL_43', 
                               options = dict(
                                   pos_x = '3.9mm',
                                   pos_y = '2.5mm',
                                   orientation = '270',
                                   north_width = '20um', 
                                   north_gap   = '10um', 
                                   south_width = '20um', 
                                   south_gap   = '10um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '44um', 
                                   finger_length   = '200um', 
                                   finger_count   = '8', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))

# gui.rebuild()
# gui.autoscale()

In [17]:
option_TL_11 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_11',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_11',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )

option_TL_12 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='Q_1',
                     pin='control'),
                 end_pin=Dict(
                     component='launch_TL_read_12',
                     pin='tie')),
              trace_width='30um',
              trace_gap='18um',qgeometry_types='poly'
             )

option_TL_13 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_13',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_13',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )

option_TL_21 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_21',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_21',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )

option_TL_22 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='Q_2',
                     pin='control'),
                 end_pin=Dict(
                     component='launch_TL_read_22',
                     pin='tie')),
              trace_width='30um',
              trace_gap='18um',qgeometry_types='poly'
             )

option_TL_23 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_23',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_23',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )

option_TL_31 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_31',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_31',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )

option_TL_32 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='Q_3',
                     pin='control'),
                 end_pin=Dict(
                     component='launch_TL_read_32',
                     pin='tie')),
              trace_width='30um',
              trace_gap='18um',qgeometry_types='poly'
             )

option_TL_33 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_33',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_33',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )

option_TL_41 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_41',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_41',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )

option_TL_42 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='Q_4',
                     pin='control'),
                 end_pin=Dict(
                     component='launch_TL_read_42',
                     pin='tie')),
              trace_width='30um',
              trace_gap='18um',qgeometry_types='poly'
             )

option_TL_43 = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL_43',
                     pin='north_end'),
                 end_pin=Dict(
                     component='launch_TL_read_43',
                     pin='tie')),
              trace_width='20um',
              trace_gap='10um',qgeometry_types='poly'
             )

TL_11 = RouteStraight(design, 'TL_11', options=option_TL_11)
TL_12 = RouteStraight(design, 'TL_12', options=option_TL_12)
TL_13 = RouteStraight(design, 'TL_13', options=option_TL_13)
TL_21 = RouteStraight(design, 'TL_21', options=option_TL_21)
TL_22 = RouteStraight(design, 'TL_22', options=option_TL_22)
TL_23 = RouteStraight(design, 'TL_23', options=option_TL_23)
TL_31 = RouteStraight(design, 'TL_31', options=option_TL_31)
TL_32 = RouteStraight(design, 'TL_32', options=option_TL_32)
TL_33 = RouteStraight(design, 'TL_33', options=option_TL_33)
TL_41 = RouteStraight(design, 'TL_41', options=option_TL_41)
TL_42 = RouteStraight(design, 'TL_42', options=option_TL_42)
TL_43 = RouteStraight(design, 'TL_43', options=option_TL_43)

# gui.rebuild()
# gui.autoscale()

## Purcell filters (X1: Readout, X3: Reset)

### T-Couplers

In [18]:
# PF-R coupler 11
coupler_length_11 = 1e-3
coupler_length_13 = 1e-3
coupler_length_21 = 1e-3
coupler_length_23 = 1e-3
coupler_length_31 = 1e-3
coupler_length_33 = 1e-3
coupler_length_41 = 1e-3
coupler_length_43 = 1e-3

T_coupler_PF_11 = LineTee_sqnl(design, 'T_coupler_PF_11', options=dict(pos_x='1.335mm', 
                                                                       pos_y='3.299mm',
                                             t_length=coupler_length_11,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = -90
                                             ))

T_coupler_PF_13 = LineTee_sqnl(design, 'T_coupler_PF_13', options=dict(pos_x='-1.335mm', 
                                                                       pos_y='2.299mm',
                                             t_length=coupler_length_13,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 90
                                             ))

T_coupler_PF_21 = LineTee_sqnl(design, 'T_coupler_PF_21', options=dict(pos_x='-2.299mm', 
                                                                       pos_y='-1.335mm',
                                             t_length=coupler_length_21,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 180
                                             ))

T_coupler_PF_23 = LineTee_sqnl(design, 'T_coupler_PF_23', options=dict(pos_x='-3.299mm', 
                                                                       pos_y='1.335mm',
                                             t_length=coupler_length_23,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 0
                                             ))

T_coupler_PF_31 = LineTee_sqnl(design, 'T_coupler_PF_31', options=dict(pos_x='1.335mm', 
                                                                       pos_y='-2.299mm',
                                             t_length=coupler_length_31,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = -90
                                             ))

T_coupler_PF_33 = LineTee_sqnl(design, 'T_coupler_PF_33', options=dict(pos_x='-1.335mm', 
                                                                       pos_y='-3.299mm',
                                             t_length=coupler_length_33,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 90
                                             ))

T_coupler_PF_41 = LineTee_sqnl(design, 'T_coupler_PF_41', options=dict(pos_x='3.299mm', 
                                                                       pos_y='-1.335mm',
                                             t_length=coupler_length_41,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 180
                                             ))

T_coupler_PF_43 = LineTee_sqnl(design, 'T_coupler_PF_43', options=dict(pos_x='2.299mm', 
                                                                       pos_y='1.335mm',
                                             t_length=coupler_length_43,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 0
                                             ))

# gui.rebuild()
# gui.autoscale()

### PF_11

In [19]:
pin_opt_PF_11 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_11',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_11_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_PF_11 = OpenToGround(design, 'PF_11_open', 
                       options=dict(
                           pos_x='1.320mm', 
                           pos_y='3.299mm',
                           width='20um',
                           gap='10um',
                           orientation=180))

pin_opt_PF_11.pin_inputs.start_pin.component = 'highC_PF_TL_11'
pin_opt_PF_11.pin_inputs.end_pin.component = 'PF_11_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_11.lead.start_straight = '400um + 90um'
pin_opt_PF_11.lead.end_straight = '0um'

# any subsequent step of the lead_start
jogsS_11 = OrderedDict()
jogsS_11[0] = ["R", '200um + 180um']
jogsS_11[1] = ["R", '1100um + 180um']
jogsS_11[2] = ["L", '0um + 180um']
jogsS_11[3] = ["L", '1930um + 180um']
jogsS_11[4] = ["R", '0um + 180um']
jogsS_11[5] = ["R", '2230um + 180um']
jogsS_11[6] = ["L", '0um + 180um']
jogsS_11[7] = ["L", '700um + 180um']

pin_opt_PF_11.lead.start_jogged_extension = jogsS_11
PF_11 = RouteFramed(design, 'PF_11', pin_opt_PF_11)

# gui.rebuild()
# gui.autoscale()

### PF_13

In [20]:
pin_opt_PF_13 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_13',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_13_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )


otg_PF_13 = OpenToGround(design, 'PF_13_open', 
                       options=dict(
                           pos_x='-1.320mm', 
                           pos_y='2.299mm',
                           width='20um',
                           gap='10um',
                           orientation=0))

pin_opt_PF_13.pin_inputs.start_pin.component = 'highC_PF_TL_13'
pin_opt_PF_13.pin_inputs.end_pin.component = 'PF_13_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_13.lead.start_straight = '400um + 90um'
pin_opt_PF_13.lead.end_straight = '0um'

# any subsequent step of the lead_start
jogsS_13 = OrderedDict()
jogsS_13[0] = ["L", '200um + 180um']
jogsS_13[1] = ["L", '1400um + 180um']
jogsS_13[2] = ["R", '0um + 180um']
jogsS_13[3] = ["R", '2130um + 180um']
jogsS_13[4] = ["L", '0um + 180um']
jogsS_13[5] = ["L", '2230um + 180um']
jogsS_13[6] = ["R", '0um + 180um']
jogsS_13[7] = ["R", '900um + 180um']

pin_opt_PF_13.lead.start_jogged_extension = jogsS_13
PF_13 = RouteFramed(design, 'PF_13', pin_opt_PF_13)

# gui.rebuild()
# gui.autoscale()

### PF_21

In [21]:
pin_opt_PF_21 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_21',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_21_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_PF_21 = OpenToGround(design, 'PF_21_open', 
                       options=dict(
                           pos_x='-3.299mm', 
                           pos_y='1.320mm',
                           width='20um',
                           gap='10um',
                           orientation=-90))

pin_opt_PF_21.pin_inputs.start_pin.component = 'highC_PF_TL_21'
pin_opt_PF_21.pin_inputs.end_pin.component = 'PF_21_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_21.lead.start_straight = '400um + 90um'
pin_opt_PF_21.lead.end_straight = '0um'

# any subsequent step of the lead_start
jogsS_21 = OrderedDict()
jogsS_21[0] = ["R", '200um + 180um']
jogsS_21[1] = ["R", '1100um + 180um']
jogsS_21[2] = ["L", '0um + 180um']
jogsS_21[3] = ["L", '1930um + 180um']
jogsS_21[4] = ["R", '0um + 180um']
jogsS_21[5] = ["R", '2230um + 180um']
jogsS_21[6] = ["L", '0um + 180um']
jogsS_21[7] = ["L", '700um + 180um']

pin_opt_PF_21.lead.start_jogged_extension = jogsS_21
PF_21 = RouteFramed(design, 'PF_21', pin_opt_PF_21)

# gui.rebuild()
# gui.autoscale()

### PF_23

In [22]:
pin_opt_PF_23 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_23',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_23_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )


otg_PF_23 = OpenToGround(design, 'PF_23_open', 
                       options=dict(
                           pos_x='-2.299mm', 
                           pos_y='-1.320mm',
                           width='20um',
                           gap='10um',
                           orientation=90))

pin_opt_PF_23.pin_inputs.start_pin.component = 'highC_PF_TL_23'
pin_opt_PF_23.pin_inputs.end_pin.component = 'PF_23_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_23.lead.start_straight = '400um + 90um'
pin_opt_PF_23.lead.end_straight = '0um'

# any subsequent step of the lead_start
jogsS_23 = OrderedDict()
jogsS_23[0] = ["L", '200um + 180um']
jogsS_23[1] = ["L", '1400um + 180um']
jogsS_23[2] = ["R", '0um + 180um']
jogsS_23[3] = ["R", '2130um + 180um']
jogsS_23[4] = ["L", '0um + 180um']
jogsS_23[5] = ["L", '2230um + 180um']
jogsS_23[6] = ["R", '0um + 180um']
jogsS_23[7] = ["R", '900um + 180um']

pin_opt_PF_23.lead.start_jogged_extension = jogsS_23
PF_23 = RouteFramed(design, 'PF_23', pin_opt_PF_23)

# gui.rebuild()
# gui.autoscale()

### PF_31

In [23]:
pin_opt_PF_31 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_31',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_31_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_PF_31 = OpenToGround(design, 'PF_31_open', 
                       options=dict(
                           pos_x='-1.320mm', 
                           pos_y='-3.299mm',
                           width='20um',
                           gap='10um',
                           orientation=0))

pin_opt_PF_31.pin_inputs.start_pin.component = 'highC_PF_TL_31'
pin_opt_PF_31.pin_inputs.end_pin.component = 'PF_31_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_31.lead.start_straight = '400um + 90um'
pin_opt_PF_31.lead.end_straight = '0um'

# any subsequent step of the lead_start
jogsS_31 = OrderedDict()
jogsS_31[0] = ["R", '200um + 180um']
jogsS_31[1] = ["R", '1100um + 180um']
jogsS_31[2] = ["L", '0um + 180um']
jogsS_31[3] = ["L", '1930um + 180um']
jogsS_31[4] = ["R", '0um + 180um']
jogsS_31[5] = ["R", '2230um + 180um']
jogsS_31[6] = ["L", '0um + 180um']
jogsS_31[7] = ["L", '700um + 180um']

pin_opt_PF_31.lead.start_jogged_extension = jogsS_31
PF_31 = RouteFramed(design, 'PF_31', pin_opt_PF_31)

# gui.rebuild()
# gui.autoscale()

### PF_33

In [24]:
pin_opt_PF_33 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_33',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_33_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )


otg_PF_33 = OpenToGround(design, 'PF_33_open', 
                       options=dict(
                           pos_x='1.320mm', 
                           pos_y='-2.299mm',
                           width='20um',
                           gap='10um',
                           orientation=180))

pin_opt_PF_33.pin_inputs.start_pin.component = 'highC_PF_TL_33'
pin_opt_PF_33.pin_inputs.end_pin.component = 'PF_33_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_33.lead.start_straight = '400um + 90um'
pin_opt_PF_33.lead.end_straight = '0um'

# any subsequent step of the lead_start
jogsS_33 = OrderedDict()
jogsS_33[0] = ["L", '200um + 180um']
jogsS_33[1] = ["L", '1400um + 180um']
jogsS_33[2] = ["R", '0um + 180um']
jogsS_33[3] = ["R", '2130um + 180um']
jogsS_33[4] = ["L", '0um + 180um']
jogsS_33[5] = ["L", '2230um + 180um']
jogsS_33[6] = ["R", '0um + 180um']
jogsS_33[7] = ["R", '900um + 180um']

pin_opt_PF_33.lead.start_jogged_extension = jogsS_33
PF_33 = RouteFramed(design, 'PF_33', pin_opt_PF_33)

# gui.rebuild()
# gui.autoscale()

### PF_41

In [25]:
pin_opt_PF_41 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_41',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_41_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_PF_41 = OpenToGround(design, 'PF_41_open', 
                       options=dict(
                           pos_x='3.299mm', 
                           pos_y='-1.320mm',
                           width='20um',
                           gap='10um',
                           orientation=90))

pin_opt_PF_41.pin_inputs.start_pin.component = 'highC_PF_TL_41'
pin_opt_PF_41.pin_inputs.end_pin.component = 'PF_41_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_41.lead.start_straight = '400um + 90um'
pin_opt_PF_41.lead.end_straight = '0um'

# any subsequent step of the lead_start
jogsS_41 = OrderedDict()
jogsS_41[0] = ["R", '200um + 180um']
jogsS_41[1] = ["R", '1100um + 180um']
jogsS_41[2] = ["L", '0um + 180um']
jogsS_41[3] = ["L", '1930um + 180um']
jogsS_41[4] = ["R", '0um + 180um']
jogsS_41[5] = ["R", '2230um + 180um']
jogsS_41[6] = ["L", '0um + 180um']
jogsS_41[7] = ["L", '700um + 180um']

pin_opt_PF_41.lead.start_jogged_extension = jogsS_41
PF_41 = RouteFramed(design, 'PF_41', pin_opt_PF_41)

# gui.rebuild()
# gui.autoscale()

### PF_43

In [26]:
pin_opt_PF_43 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='highC_PF_TL_43',
                                    pin='south_end'),
                                    end_pin=Dict(
                                    component='PF_43_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )


otg_PF_43 = OpenToGround(design, 'PF_43_open', 
                       options=dict(
                           pos_x='2.299mm', 
                           pos_y='1.320mm',
                           width='20um',
                           gap='10um',
                           orientation=-90))

pin_opt_PF_43.pin_inputs.start_pin.component = 'highC_PF_TL_43'
pin_opt_PF_43.pin_inputs.end_pin.component = 'PF_43_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_PF_43.lead.start_straight = '400um + 90um'
pin_opt_PF_43.lead.end_straight = '0um'

# any subsequent step of the lead_start
jogsS_43 = OrderedDict()
jogsS_43[0] = ["L", '200um + 180um']
jogsS_43[1] = ["L", '1400um + 180um']
jogsS_43[2] = ["R", '0um + 180um']
jogsS_43[3] = ["R", '2130um + 180um']
jogsS_43[4] = ["L", '0um + 180um']
jogsS_43[5] = ["L", '2230um + 180um']
jogsS_43[6] = ["R", '0um + 180um']
jogsS_43[7] = ["R", '900um + 180um']

pin_opt_PF_43.lead.start_jogged_extension = jogsS_43
PF_43 = RouteFramed(design, 'PF_43', pin_opt_PF_43)

# gui.rebuild()
# gui.autoscale()

## Readout resonators (X1: Readout, X3: Reset)

### T-Couplers

In [27]:
# R-PF coupler 11
coupler_length_11 = 1e-3
coupler_length_13 = 1e-3
coupler_length_21 = 1e-3
coupler_length_23 = 1e-3
coupler_length_31 = 1e-3
coupler_length_33 = 1e-3
coupler_length_41 = 1e-3
coupler_length_43 = 1e-3

T_coupler_R_11 = LineTee_sqnl(design, 'T_coupler_R_11', options=dict(pos_x='1.280mm', 
                                                                       pos_y='3.299mm',
                                             t_length=coupler_length_11,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 90
                                             ))

T_coupler_R_13 = LineTee_sqnl(design, 'T_coupler_R_13', options=dict(pos_x='-1.280mm', 
                                                                       pos_y='2.299mm',
                                             t_length=coupler_length_13,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = -90
                                             ))

T_coupler_R_21 = LineTee_sqnl(design, 'T_coupler_R_21', options=dict(pos_x='-3.299mm', 
                                                                       pos_y='1.280mm',
                                             t_length=coupler_length_21,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 180
                                             ))

T_coupler_R_23 = LineTee_sqnl(design, 'T_coupler_R_23', options=dict(pos_x='-2.299mm', 
                                                                       pos_y='-1.280mm',
                                             t_length=coupler_length_23,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 0
                                             ))

T_coupler_R_31 = LineTee_sqnl(design, 'T_coupler_R_31', options=dict(pos_x='-1.280mm', 
                                                                       pos_y='-3.299mm',
                                             t_length=coupler_length_31,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = -90
                                             ))

T_coupler_R_33 = LineTee_sqnl(design, 'T_coupler_R_33', options=dict(pos_x='1.280mm', 
                                                                       pos_y='-2.299mm',
                                             t_length=coupler_length_33,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 90
                                             ))

T_coupler_R_41 = LineTee_sqnl(design, 'T_coupler_R_41', options=dict(pos_x='3.299mm', 
                                                                       pos_y='-1.280mm',
                                             t_length=coupler_length_41,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 0
                                             ))

T_coupler_R_43 = LineTee_sqnl(design, 'T_coupler_R_43', options=dict(pos_x='2.299mm', 
                                                                       pos_y='1.280mm',
                                             t_length=coupler_length_43,
                                             prime_width = '10um',
                                             prime_gap = '10um',
                                             second_width = '20um',
                                             second_gap = '20um',
                                             C_box_width = '60um',
                                             C_box_gap = '10um',
                                             C_box_height = '20um',
                                             C_gap = '30um',
                                             orientation = 180
                                             ))

# gui.rebuild()
# gui.autoscale()

### R_11

In [28]:
pin_opt_R_11 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_1',
                                    pin='readout1'),
                                end_pin=Dict(
                                    component='R_11_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_R_11 = OpenToGround(design, 'R_11_open', 
                       options=dict(
                           pos_x='1.296mm', 
                           pos_y='3.299mm',
                           width='20um',
                           gap='10um',
                           orientation=0))

pin_opt_R_11.pin_inputs.start_pin.component = 'Q_1'
pin_opt_R_11.pin_inputs.end_pin.component = 'R_11_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_11.lead.start_straight = '0um + 90um'
pin_opt_R_11.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_11 = OrderedDict()
jogsS_11[0] = ["L", '1400um + 180um']
jogsS_11[1] = ["R", '0um + 180um']
jogsS_11[2] = ["R", '2600um + 180um']
jogsS_11[3] = ["L", '0um + 180um']
jogsS_11[4] = ["L", '2700um + 180um']
jogsS_11[5] = ["R", '0um + 180um']
jogsS_11[6] = ["R", '700um + 180um']

pin_opt_R_11.lead.start_jogged_extension = jogsS_11
R_11 = RouteFramed(design, 'R_11', pin_opt_R_11)

# gui.rebuild()
# gui.autoscale()

### R_13

In [29]:
pin_opt_R_13 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_1',
                                    pin='readout2'),
                                end_pin=Dict(
                                    component='R_13_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_R_13 = OpenToGround(design, 'R_13_open', 
                       options=dict(
                           pos_x='-1.296mm', 
                           pos_y='2.299mm',
                           width='20um',
                           gap='10um',
                           orientation=180))

pin_opt_R_13.pin_inputs.start_pin.component = 'Q_1'
pin_opt_R_13.pin_inputs.end_pin.component = 'R_13_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_13.lead.start_straight = '0um + 90um'
pin_opt_R_13.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_13 = OrderedDict()
jogsS_13[0] = ["R", '1700um + 180um']
jogsS_13[1] = ["L", '0um + 180um']
jogsS_13[2] = ["L", '2900um + 180um']
jogsS_13[3] = ["R", '0um + 180um']
jogsS_13[4] = ["R", '2700um + 180um']
jogsS_13[5] = ["L", '0um + 180um']
jogsS_13[6] = ["L", '700um + 180um']

pin_opt_R_13.lead.start_jogged_extension = jogsS_13
R_13 = RouteFramed(design, 'R_13', pin_opt_R_13)

# gui.rebuild()
# gui.autoscale()

### R_21

In [30]:
pin_opt_R_21 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_2',
                                    pin='readout1'),
                                end_pin=Dict(
                                    component='R_21_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_R_21 = OpenToGround(design, 'R_21_open', 
                       options=dict(
                           pos_x='-3.299mm', 
                           pos_y='1.296mm',
                           width='20um',
                           gap='10um',
                           orientation=90))

pin_opt_R_21.pin_inputs.start_pin.component = 'Q_2'
pin_opt_R_21.pin_inputs.end_pin.component = 'R_21_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_21.lead.start_straight = '0um + 90um'
pin_opt_R_21.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_21 = OrderedDict()
jogsS_21[0] = ["L", '1400um + 180um']
jogsS_21[1] = ["R", '0um + 180um']
jogsS_21[2] = ["R", '2600um + 180um']
jogsS_21[3] = ["L", '0um + 180um']
jogsS_21[4] = ["L", '2700um + 180um']
jogsS_21[5] = ["R", '0um + 180um']
jogsS_21[6] = ["R", '700um + 180um']

pin_opt_R_21.lead.start_jogged_extension = jogsS_21
R_21 = RouteFramed(design, 'R_21', pin_opt_R_21)

# gui.rebuild()
# gui.autoscale()

### R_23

In [31]:
pin_opt_R_23 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_2',
                                    pin='readout2'),
                                end_pin=Dict(
                                    component='R_23_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_R_23 = OpenToGround(design, 'R_23_open', 
                       options=dict(
                           pos_x='-2.299mm', 
                           pos_y='-1.296mm',
                           width='20um',
                           gap='10um',
                           orientation=-90))

pin_opt_R_23.pin_inputs.start_pin.component = 'Q_2'
pin_opt_R_23.pin_inputs.end_pin.component = 'R_23_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_23.lead.start_straight = '0um + 90um'
pin_opt_R_23.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_23 = OrderedDict()
jogsS_23[0] = ["R", '1700um + 180um']
jogsS_23[1] = ["L", '0um + 180um']
jogsS_23[2] = ["L", '2900um + 180um']
jogsS_23[3] = ["R", '0um + 180um']
jogsS_23[4] = ["R", '2700um + 180um']
jogsS_23[5] = ["L", '0um + 180um']
jogsS_23[6] = ["L", '700um + 180um']

pin_opt_R_23.lead.start_jogged_extension = jogsS_23
R_23 = RouteFramed(design, 'R_23', pin_opt_R_23)

# gui.rebuild()
# gui.autoscale()

### R_31

In [32]:
pin_opt_R_31 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_3',
                                    pin='readout1'),
                                end_pin=Dict(
                                    component='R_31_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_R_31 = OpenToGround(design, 'R_31_open', 
                       options=dict(
                           pos_x='-1.296mm', 
                           pos_y='-3.299mm',
                           width='20um',
                           gap='10um',
                           orientation=180))

pin_opt_R_31.pin_inputs.start_pin.component = 'Q_3'
pin_opt_R_31.pin_inputs.end_pin.component = 'R_31_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_31.lead.start_straight = '0um + 90um'
pin_opt_R_31.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_31 = OrderedDict()
jogsS_31[0] = ["L", '1400um + 180um']
jogsS_31[1] = ["R", '0um + 180um']
jogsS_31[2] = ["R", '2600um + 180um']
jogsS_31[3] = ["L", '0um + 180um']
jogsS_31[4] = ["L", '2700um + 180um']
jogsS_31[5] = ["R", '0um + 180um']
jogsS_31[6] = ["R", '700um + 180um']

pin_opt_R_31.lead.start_jogged_extension = jogsS_31
R_31 = RouteFramed(design, 'R_31', pin_opt_R_31)

# gui.rebuild()
# gui.autoscale()

### R_33

In [33]:
pin_opt_R_33 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_3',
                                    pin='readout2'),
                                end_pin=Dict(
                                    component='R_33_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_R_33 = OpenToGround(design, 'R_33_open', 
                       options=dict(
                           pos_x='1.296mm', 
                           pos_y='-2.299mm',
                           width='20um',
                           gap='10um',
                           orientation=0))

pin_opt_R_33.pin_inputs.start_pin.component = 'Q_3'
pin_opt_R_33.pin_inputs.end_pin.component = 'R_33_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_33.lead.start_straight = '0um + 90um'
pin_opt_R_33.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_33 = OrderedDict()
jogsS_33[0] = ["R", '1700um + 180um']
jogsS_33[1] = ["L", '0um + 180um']
jogsS_33[2] = ["L", '2900um + 180um']
jogsS_33[3] = ["R", '0um + 180um']
jogsS_33[4] = ["R", '2700um + 180um']
jogsS_33[5] = ["L", '0um + 180um']
jogsS_33[6] = ["L", '700um + 180um']

pin_opt_R_33.lead.start_jogged_extension = jogsS_33
R_33 = RouteFramed(design, 'R_33', pin_opt_R_33)

# gui.rebuild()
# gui.autoscale()

### R_41

In [34]:
pin_opt_R_41 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_4',
                                    pin='readout1'),
                                end_pin=Dict(
                                    component='R_41_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_R_41 = OpenToGround(design, 'R_41_open', 
                       options=dict(
                           pos_x='3.299mm', 
                           pos_y='-1.296mm',
                           width='20um',
                           gap='10um',
                           orientation=-90))

pin_opt_R_41.pin_inputs.start_pin.component = 'Q_4'
pin_opt_R_41.pin_inputs.end_pin.component = 'R_41_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_41.lead.start_straight = '0um + 90um'
pin_opt_R_41.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_41 = OrderedDict()
jogsS_41[0] = ["L", '1400um + 180um']
jogsS_41[1] = ["R", '0um + 180um']
jogsS_41[2] = ["R", '2600um + 180um']
jogsS_41[3] = ["L", '0um + 180um']
jogsS_41[4] = ["L", '2700um + 180um']
jogsS_41[5] = ["R", '0um + 180um']
jogsS_41[6] = ["R", '700um + 180um']

pin_opt_R_41.lead.start_jogged_extension = jogsS_41
R_41 = RouteFramed(design, 'R_41', pin_opt_R_41)

# gui.rebuild()
# gui.autoscale()

### R_43

In [ ]:
pin_opt_R_43 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_4',
                                    pin='readout2'),
                                end_pin=Dict(
                                    component='R_43_open',
                                    pin='open')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um',
               hfss_wire_bonds = True,
               wb_threshold = '10um',
               wb_offset ='10um'
              )

otg_R_43 = OpenToGround(design, 'R_43_open', 
                       options=dict(
                           pos_x='2.299mm', 
                           pos_y='1.296mm',
                           width='20um',
                           gap='10um',
                           orientation=90))

pin_opt_R_43.pin_inputs.start_pin.component = 'Q_4'
pin_opt_R_43.pin_inputs.end_pin.component = 'R_43_open'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_R_43.lead.start_straight = '0um + 90um'
pin_opt_R_43.lead.end_straight = '50um'

# any subsequent step of the lead_start
jogsS_43 = OrderedDict()
jogsS_43[0] = ["R", '1700um + 180um']
jogsS_43[1] = ["L", '0um + 180um']
jogsS_43[2] = ["L", '2900um + 180um']
jogsS_43[3] = ["R", '0um + 180um']
jogsS_43[4] = ["R", '2700um + 180um']
jogsS_43[5] = ["L", '0um + 180um']
jogsS_43[6] = ["L", '700um + 180um']

pin_opt_R_43.lead.start_jogged_extension = jogsS_43
R_43 = RouteFramed(design, 'R_43', pin_opt_R_43)

gui.rebuild()
gui.autoscale()

01:07PM 44s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QWindowsNativeFileDialogBase::selectNameFilter: Invalid parameter '*.metal.py' not found in 'All Files (*)'.

01:07PM 44s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QWindowsNativeFileDialogBase::selectNameFilter: Invalid parameter '*.metal.py' not found in 'All Files (*)'.

01:08PM 09s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QWindowsNativeFileDialogBase::selectNameFilter: Invalid parameter '*.gds' not found in 'All Files (*)'.

01:08PM 09s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QWindowsNativeFileDialogBase::selectNameFilter: Invalid parameter '*.gds' not found in 'All Files (*)'.

01:09PM 05s WARNING [_import_junction_gds_file]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"c:\Users\Luxcuse\Documents\GitHub\resources".
01:09PM 05s WARN